# Make forcing file order .csv
GRUs/HRUs in SUMMA's `attributes.nc` and `forcing.nc` need to be in the same order. This code generates a remapping `.csv` file that contains the required order for the ERA5 forcing files (North America domain [-180,-50,5,85] [lon,lon,lat,lat]) so that they match the attributes file for this domain.

File locations:
- Intersection shapefile between ERA5 and Merit Hydro: `C:/Globus endpint/ERA5_NA/ERA5_NA_merit_intersect/merit_hydro_basins_and_coastal_hillslopes_merged_NA_ERA5_ERA5elev.shp`
- Attributes file: `C:/Globus endpoint/ERA5_NA/SUMMA_files/attributes_na_catchment_initial.nc`

Source: this code is based on code shared by Shervan Gharari (Nov 2020).

## Step 0: modules, packages and functions

In [2]:
import glob
import time
import geopandas as gpd
import netCDF4 as nc4
import numpy as np
import pandas as pd
import xarray as xr
#from shapely.geometry import Polygon
from datetime import datetime
#import matplotlib.pyplot as plt

In [22]:
def mapped_lat_lon(case,
                   nc_name,
                   lon_var,
                   lon_dim,
                   lat_var,
                   lat_dim,
                   time_dim,
                   var_name):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/candex
    @ author's email id:       sh.gharari@gmail.com
    @license:                  Apache2

    This function read the varibale lat and lon and creates a 2D or 1D array based on case
    if case 1, 3-dimensional variable with 1-dimentional lat and lon:
    The function creates a 2D array of lat and lon that matches the varibale
    if case 2, 3-dimensional variable with 2-dimentional lat and lon
    The function reads the 2D lat and lon variables
    if case 3, 2-dimensional variable with 1-dimentional lat and lon
    The function returns the 1D variable of lat and lon

    Arguments
    ---------
    case:value [1,]
            1 is for 3-dimensional variable with 1-dimentional lat and lon
            2 is for 3-dimensional variable with 2-dimentional lat and lon
            3 is for 2-dimensional variable with 1-dimentional lat and lon
    nc_name: str, name of the nc file including path
    lon_var: name of variable lon
    lon_dim: name of dimension lon
    lat_var: name of variable lat
    lat_dim: name of dimension lat
    time_dim: name of dimension time
    var_name: name of variable to be read from nc file

    Returns
    -------
    latmap: a numpy array for the lat of the data
    lonmap: a numpy array for the lon of the data
    """

    if case == 1:  # regular lat/lon, the data is provided in 2D and lat and lon values in 1D
        ds = xr.open_dataset(nc_name) # open the data set
        dims = ds[var_name].dims # get the dimension of the variable to be read
        dims = list(dims) # pass that to a list
        dims.remove(time_dim) # remove the time dimension
        lat = np.array(ds[lat_var]) # read  1D lat
        lon = np.array(ds[lon_var]) # read 1D lon
        if dims[0] == lon_dim: # if lon is first dimension
            latmap = np.zeros((len(lon), len(lat)))
            lonmap = np.zeros((len(lon), len(lat)))
            for i in np.arange(len(lat)):
                lonmap [:,i] = lon
            for i in np.arange(len(lon)):
                latmap [i,:] = lat
        else: # if lat is first dimension
            latmap = np.zeros((len(lat), len(lon)))
            lonmap = np.zeros((len(lat), len(lon)))
            for i in np.arange(len(lat)):
                lonmap [i,:] = lon
            for i in np.arange(len(lon)):
                latmap [:,i] = lat

    if case == 2:  # 2D lat/lon, the data is provided in 2D and lat and lon values in 2D
        ds = xr.open_dataset(nc_name)
        latmap = np.array(ds[lat_var])
        lonmap = np.array(ds[lon_var])

    if case == 3: # non regular 1D data 1D lat/lon
        latmap = np.array(ds[lat_var])
        lonmap = np.array(ds[lon_var])

    return latmap, lonmap

In [23]:
def lat_lon_to_index(lat_source,
                     lon_source,
                     lat_mapped,
                     lon_mapped):
    """
    @ author:                  Shervan Gharari
    @ Github:                  https://github.com/ShervanGharari/candex
    @ author's email id:       sh.gharari@gmail.com
    @license:                  Apache2

    This function get the mapped lat and lon from data created by mapped_lat_lon function and find
    the index for the remap dataframe to that

    Arguments
    ---------
    lat_source: 1D numpy array of lat_source from remap data frame
    lon_source: 1D numpy array of lon_source from remap data frame
    lat_mapped: 2D or 1D numpy array of data lat values
    lon_mapped: 2D or 1D numpy array of data lon values

    Returns
    -------
    rows: a 1D numpy array indexing the row of data to be read
    cols: a 1D numpy array indexing the col of data to be read
    """

    # create the np arrays
    rows = np.zeros(len(lat_source))
    cols = np.zeros(len(lon_source))

    # loop to find the rows and colomns
    for i in np.arange(len(lat_source)):
        lat_lon_value_diff = abs(lat_mapped - lat_source[i])+abs(lon_mapped - lon_source[i])
        row, col = np.where(lat_lon_value_diff == lat_lon_value_diff.min())
        rows [i] = row[0]
        cols [i] = col[0]

    return rows, cols

## Step 1: reading the remap shapefile and prepare the indexing

In [46]:
# File locations
file_shp = 'C:\Globus endpoint\ERA5_NA\ERA5_NA_merit_intersect\merit_hydro_basins_and_coastal_hillslopes_merged_NA_ERA5_ERA5elev.shp'
file_att = 'C:/Globus endpoint/ERA5_NA/SUMMA_files/attributes_na_catchment_initial.nc'
file_for = 'C:\Globus endpoint\ERA5_NA\ERA5_NA_forcing_example\ERA5_NA_197901.nc'

In [13]:
# Load the shape
shp = gpd.read_file(file_shp)

In [14]:
# drop the unnecessary colomn
shp = shp.drop(columns=['S_1_area', 'AS1', 'IDS1', 
                        'AS2', 'IDS2','AINT','AP1','AP2','AP2N'])

In [15]:
# check what's left
shp.columns

Index(['S_1_COMID', 'S_1_elev_m', 'S_1_lon_c', 'S_1_lat_c', 'S_2_ID',
       'S_2_lat', 'S_2_lon', 'AP1N', 'S_2_elev_m', 'geometry'],
      dtype='object')

In [16]:
# rename essential filed so that the dataframe can work in the
dict_rename = {"S_1_COMID"  : "ID_target",  # 'hruId' that is used as SUMMA's computational units ID
               "S_1_elev_m" : "MERIT_ele",  # Elevation of the HRU (catchment)  [m]
               "S_1_lon_c"  : "lon_target", # Longitude of the HRU (catchment)
               "S_1_lat_c"  : "lat_target", # Latitude of the HRU (catchment)
               "S_2_ID"     : "ID_source",  # ERA5 grid ID, not used
               "S_2_lat"    : "lat_source", # Latitude of forcing grid to be read
               "S_2_lon"    : "lon_source", # Longitude of forcing grid to be read
               "AP1N"       : "weight",     # Weight of each ERA5 grid in subbasin (i.e. part of the HRU that falls in a given ERA5 grid cell)
               "S_2_elev_m" : "ERA5_ele"}   # Elevation of the ERA5 data point [m]

shp = shp.rename(columns=dict_rename) 

In [19]:
# Sort the dataframe based on the ID_target. This ensures that all parts of each HRUs are side-by-side
shp = shp.sort_values(by=['ID_target'])

In [88]:
# Reset the indices
shp = shp.reset_index(drop=True)

In [89]:
# check what's left
shp

,ID_target,MERIT_ele,lon_target,lat_target,ID_source,lat_source,lon_source,weight,ERA5_ele,geometry,rows,cols,order
0,71000001,279.200840,-98.114080,59.991349,105389.0,60.00,-98.00,0.655095,283,"MULTIPOLYGON (((-98.12500 60.05292, -98.12458 ...",100.0,328.0,1
1,71000001,279.200840,-98.114080,59.991349,105068.0,60.00,-98.25,0.344905,296,"POLYGON ((-98.14542 60.06375, -98.13792 60.063...",100.0,327.0,1
2,71000002,279.792634,-98.115715,60.091655,105067.0,60.25,-98.25,0.152988,275,"POLYGON ((-98.16542 60.12500, -98.16542 60.126...",99.0,327.0,1
3,71000002,279.792634,-98.115715,60.091655,105388.0,60.25,-98.00,0.088848,250,"MULTIPOLYGON (((-98.12500 60.19125, -98.12292 ...",99.0,328.0,1
4,71000002,279.792634,-98.115715,60.091655,105068.0,60.00,-98.25,0.258413,296,"MULTIPOLYGON (((-98.18125 60.08708, -98.18208 ...",100.0,327.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
964180,83034687,29.851234,-119.864094,69.335548,77104.0,69.25,-120.00,0.230905,287,"POLYGON ((-119.90208 69.34375, -119.90208 69.3...",63.0,240.0,1
964181,83034688,104.512570,-119.573274,69.283675,77746.0,69.25,-119.50,0.700302,175,"POLYGON ((-119.62500 69.31958, -119.61875 69.3...",63.0,242.0,1
964182,83034688,104.512570,-119.573274,69.283675,78067.0,69.25,-119.25,0.000679,117,"POLYGON ((-119.37500 69.29875, -119.37458 69.2...",63.0,243.0,1
964183,83034688,104.512570,-119.573274,69.283675,77425.0,69.25,-119.75,0.299019,232,"POLYGON ((-119.82292 69.34625, -119.82292 69.3...",63.0,241.0,1


In [38]:
# Call the first function to recreate the lat lon from the data, generates 2D matrices for lat and lon
# Note: requires an example forcing file
lat_mapped, lon_mapped = mapped_lat_lon(1,           # case selection
                                        file_for,    # name of the forcing sample file
                                       'longitude',  # name of the longitude variable in the sample file
                                       'longitude',  # name of the lon dimension in sample
                                       'latitude',   # name of the lat variable in sample
                                       'latitude',   # name of the lat dimension in sample
                                       'time',       # name of the time dimension in sample
                                       'airtemp')    # name of the forcing variable to use (doesn't really matter which one)

In [40]:
# Finding the index rows cols and pass the to the dataframe
rows, cols = lat_lon_to_index(np.array(shp['lat_source']),
                              np.array(shp['lon_source']),
                              lat_mapped,
                              lon_mapped)

In [42]:
# Add the newfound indices to the shp dataframe
shp['rows'] = rows
shp['cols'] = cols

## Step 2: Find the hruId order in the attributes.nc file

In [47]:
# Open the attributes file
ncid_att = nc4.Dataset(file_att) 

In [48]:
# Make a new dataframe to store the indexing in
hru_lat_lon = pd.DataFrame()
hru_lat_lon['ID_target']  = ncid_att['hruId'][:]     # get the hruID
hru_lat_lon['lon_target'] = ncid_att['longitude'][:] # get the lon
hru_lat_lon['lat_target'] = ncid_att['latitude'][:]  # get the lat

### Re-order the shp based on the existing hruId in the dataframe

In [50]:
# Add the order column
shp['order'] = 1 

In [150]:
# Check what we currently have
shp

,ID_target,MERIT_ele,lon_target,lat_target,ID_source,lat_source,lon_source,weight,ERA5_ele,geometry,rows,cols,order
0,71000001,279.200840,-98.114080,59.991349,105389.0,60.00,-98.00,0.655095,283,"MULTIPOLYGON (((-98.12500 60.05292, -98.12458 ...",100.0,328.0,1
1,71000001,279.200840,-98.114080,59.991349,105068.0,60.00,-98.25,0.344905,296,"POLYGON ((-98.14542 60.06375, -98.13792 60.063...",100.0,327.0,1
2,71000002,279.792634,-98.115715,60.091655,105067.0,60.25,-98.25,0.152988,275,"POLYGON ((-98.16542 60.12500, -98.16542 60.126...",99.0,327.0,1
3,71000002,279.792634,-98.115715,60.091655,105388.0,60.25,-98.00,0.088848,250,"MULTIPOLYGON (((-98.12500 60.19125, -98.12292 ...",99.0,328.0,1
4,71000002,279.792634,-98.115715,60.091655,105068.0,60.00,-98.25,0.258413,296,"MULTIPOLYGON (((-98.18125 60.08708, -98.18208 ...",100.0,327.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
964180,83034687,29.851234,-119.864094,69.335548,77104.0,69.25,-120.00,0.230905,287,"POLYGON ((-119.90208 69.34375, -119.90208 69.3...",63.0,240.0,1
964181,83034688,104.512570,-119.573274,69.283675,77746.0,69.25,-119.50,0.700302,175,"POLYGON ((-119.62500 69.31958, -119.61875 69.3...",63.0,242.0,1
964182,83034688,104.512570,-119.573274,69.283675,78067.0,69.25,-119.25,0.000679,117,"POLYGON ((-119.37500 69.29875, -119.37458 69.2...",63.0,243.0,1
964183,83034688,104.512570,-119.573274,69.283675,77425.0,69.25,-119.75,0.299019,232,"POLYGON ((-119.82292 69.34625, -119.82292 69.3...",63.0,241.0,1


In [151]:
# Set ID_target as the index
shp = shp.set_index('ID_target')
shp

,MERIT_ele,lon_target,lat_target,ID_source,lat_source,lon_source,weight,ERA5_ele,geometry,rows,cols,order
ID_target,,,,,,,,,,,,
71000001,279.200840,-98.114080,59.991349,105389.0,60.00,-98.00,0.655095,283,"MULTIPOLYGON (((-98.12500 60.05292, -98.12458 ...",100.0,328.0,1
71000001,279.200840,-98.114080,59.991349,105068.0,60.00,-98.25,0.344905,296,"POLYGON ((-98.14542 60.06375, -98.13792 60.063...",100.0,327.0,1
71000002,279.792634,-98.115715,60.091655,105067.0,60.25,-98.25,0.152988,275,"POLYGON ((-98.16542 60.12500, -98.16542 60.126...",99.0,327.0,1
71000002,279.792634,-98.115715,60.091655,105388.0,60.25,-98.00,0.088848,250,"MULTIPOLYGON (((-98.12500 60.19125, -98.12292 ...",99.0,328.0,1
71000002,279.792634,-98.115715,60.091655,105068.0,60.00,-98.25,0.258413,296,"MULTIPOLYGON (((-98.18125 60.08708, -98.18208 ...",100.0,327.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
83034687,29.851234,-119.864094,69.335548,77104.0,69.25,-120.00,0.230905,287,"POLYGON ((-119.90208 69.34375, -119.90208 69.3...",63.0,240.0,1
83034688,104.512570,-119.573274,69.283675,77746.0,69.25,-119.50,0.700302,175,"POLYGON ((-119.62500 69.31958, -119.61875 69.3...",63.0,242.0,1
83034688,104.512570,-119.573274,69.283675,78067.0,69.25,-119.25,0.000679,117,"POLYGON ((-119.37500 69.29875, -119.37458 69.2...",63.0,243.0,1


In [153]:
# Convert the ID_target index to a multi-index
shp = shp.assign(index=shp.groupby(level=0).cumcount()).set_index('index',append=True)
shp

MERIT_ele  lon_target  lat_target  ID_source  lat_source  \
ID_target index                                                              
71000001  0      279.200840  -98.114080   59.991349   105389.0       60.00   
          1      279.200840  -98.114080   59.991349   105068.0       60.00   
71000002  0      279.792634  -98.115715   60.091655   105067.0       60.25   
          1      279.792634  -98.115715   60.091655   105388.0       60.25   
          2      279.792634  -98.115715   60.091655   105068.0       60.00   
...                     ...         ...         ...        ...         ...   
83034687  1       29.851234 -119.864094   69.335548    77104.0       69.25   
83034688  0      104.512570 -119.573274   69.283675    77746.0       69.25   
          1      104.512570 -119.573274   69.283675    78067.0       69.25   
          2      104.512570 -119.573274   69.283675    77425.0       69.25   
83034689  0        9.828125 -119.368333   69.292500    78067.0       69.25   

                 lon_source    weight  ERA5_ele  \
ID_target index                                   
71000001  0          -98.00  0.655095       283   
          1          -98.25  0.344905       296   
71000002  0          -98.25  0.152988       275   
          1          -98.00  0.088848       250   
          2          -98.25  0.258413       296   
...                     ...       ...       ...   
83034687  1         -120.00  0.230905       287   
83034688  0         -119.50  0.700302       175   
          1         -119.25  0.000679       117   
          2         -119.75  0.299019       232   
83034689  0         -119.25  1.000000       117   

                                                          geometry   rows  \
ID_target index                                                             
71000001  0      MULTIPOLYGON (((-98.12500 60.05292, -98.12458 ...  100.0   
          1      POLYGON ((-98.14542 60.06375, -98.13792 60.063...  100.0   
71000002  0      POLYGON ((-98.16542 60.12500, -98.16542 60.126...   99.0   
          1      MULTIPOLYGON (((-98.12500 60.19125, -98.12292 ...   99.0   
          2      MULTIPOLYGON (((-98.18125 60.08708, -98.18208 ...  100.0   
...                                                            ...    ...   
83034687  1      POLYGON ((-119.90208 69.34375, -119.90208 69.3...   63.0   
83034688  0      POLYGON ((-119.62500 69.31958, -119.61875 69.3...   63.0   
          1      POLYGON ((-119.37500 69.29875, -119.37458 69.2...   63.0   
          2      POLYGON ((-119.82292 69.34625, -119.82292 69.3...   63.0   
83034689  0      POLYGON ((-119.37125 69.29458, -119.37125 69.2...   63.0   

                  cols  order  
ID_target index                
71000001  0      328.0      1  
          1      327.0      1  
71000002  0      327.0      1  
          1      328.0      1  
          2      327.0      1  
...                ...    ...  
83034687  1      240.0      1  
83034688  0      242.0      1  
          1      243.0      1  
          2      241.0      1  
83034689  0      243.0      1  

[964185 rows x 12 columns]

In [154]:
# Sort based on the order of ID_target in the attributes
shp = shp.reindex(index=hru_lat_lon['ID_target'], level=0)
shp

MERIT_ele  lon_target  lat_target  ID_source  lat_source  \
ID_target index                                                              
71000092  0      402.158863 -102.334034   58.984292    99936.0       59.00   
          1      402.158863 -102.334034   58.984292    99615.0       59.00   
71007360  0      302.931320 -100.121819   56.987527   102512.0       57.00   
          1      302.931320 -100.121819   56.987527   102833.0       57.00   
71011339  0      270.429858  -98.415879   56.020979   104763.0       56.00   
...                     ...         ...         ...        ...         ...   
83034687  1       29.851234 -119.864094   69.335548    77104.0       69.25   
83034688  0      104.512570 -119.573274   69.283675    77746.0       69.25   
          1      104.512570 -119.573274   69.283675    78067.0       69.25   
          2      104.512570 -119.573274   69.283675    77425.0       69.25   
83034689  0        9.828125 -119.368333   69.292500    78067.0       69.25   

                 lon_source    weight  ERA5_ele  \
ID_target index                                   
71000092  0         -102.25  0.914826       400   
          1         -102.50  0.085174       409   
71007360  0         -100.25  0.461395       316   
          1         -100.00  0.538605       310   
71011339  0          -98.50  0.877963       269   
...                     ...       ...       ...   
83034687  1         -120.00  0.230905       287   
83034688  0         -119.50  0.700302       175   
          1         -119.25  0.000679       117   
          2         -119.75  0.299019       232   
83034689  0         -119.25  1.000000       117   

                                                          geometry   rows  \
ID_target index                                                             
71000092  0      POLYGON ((-102.34292 59.01875, -102.33958 59.0...  104.0   
          1      POLYGON ((-102.37500 58.97125, -102.37708 58.9...  104.0   
71007360  0      POLYGON ((-100.12500 56.92542, -100.12542 56.9...  112.0   
          1      POLYGON ((-100.10875 57.05625, -100.10458 57.0...  112.0   
71011339  0      POLYGON ((-98.47625 56.06708, -98.47542 56.067...  116.0   
...                                                            ...    ...   
83034687  1      POLYGON ((-119.90208 69.34375, -119.90208 69.3...   63.0   
83034688  0      POLYGON ((-119.62500 69.31958, -119.61875 69.3...   63.0   
          1      POLYGON ((-119.37500 69.29875, -119.37458 69.2...   63.0   
          2      POLYGON ((-119.82292 69.34625, -119.82292 69.3...   63.0   
83034689  0      POLYGON ((-119.37125 69.29458, -119.37125 69.2...   63.0   

                  cols  order  
ID_target index                
71000092  0      311.0      1  
          1      310.0      1  
71007360  0      319.0      1  
          1      320.0      1  
71011339  0      326.0      1  
...                ...    ...  
83034687  1      240.0      1  
83034688  0      242.0      1  
          1      243.0      1  
          2      241.0      1  
83034689  0      243.0      1  

[964185 rows x 12 columns]

In [158]:
# Reset the multi-index to a single index
shp = shp.reset_index() 
shp

,ID_target,index,MERIT_ele,lon_target,lat_target,ID_source,lat_source,lon_source,weight,ERA5_ele,geometry,rows,cols,order
0,71000092,0,402.158863,-102.334034,58.984292,99936.0,59.00,-102.25,0.914826,400,"POLYGON ((-102.34292 59.01875, -102.33958 59.0...",104.0,311.0,1
1,71000092,1,402.158863,-102.334034,58.984292,99615.0,59.00,-102.50,0.085174,409,"POLYGON ((-102.37500 58.97125, -102.37708 58.9...",104.0,310.0,1
2,71007360,0,302.931320,-100.121819,56.987527,102512.0,57.00,-100.25,0.461395,316,"POLYGON ((-100.12500 56.92542, -100.12542 56.9...",112.0,319.0,1
3,71007360,1,302.931320,-100.121819,56.987527,102833.0,57.00,-100.00,0.538605,310,"POLYGON ((-100.10875 57.05625, -100.10458 57.0...",112.0,320.0,1
4,71011339,0,270.429858,-98.415879,56.020979,104763.0,56.00,-98.50,0.877963,269,"POLYGON ((-98.47625 56.06708, -98.47542 56.067...",116.0,326.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964180,83034687,1,29.851234,-119.864094,69.335548,77104.0,69.25,-120.00,0.230905,287,"POLYGON ((-119.90208 69.34375, -119.90208 69.3...",63.0,240.0,1
964181,83034688,0,104.512570,-119.573274,69.283675,77746.0,69.25,-119.50,0.700302,175,"POLYGON ((-119.62500 69.31958, -119.61875 69.3...",63.0,242.0,1
964182,83034688,1,104.512570,-119.573274,69.283675,78067.0,69.25,-119.25,0.000679,117,"POLYGON ((-119.37500 69.29875, -119.37458 69.2...",63.0,243.0,1
964183,83034688,2,104.512570,-119.573274,69.283675,77425.0,69.25,-119.75,0.299019,232,"POLYGON ((-119.82292 69.34625, -119.82292 69.3...",63.0,241.0,1


In [159]:
# Remove the 'index' column
del shp['index']
shp

,ID_target,MERIT_ele,lon_target,lat_target,ID_source,lat_source,lon_source,weight,ERA5_ele,geometry,rows,cols,order
0,71000092,402.158863,-102.334034,58.984292,99936.0,59.00,-102.25,0.914826,400,"POLYGON ((-102.34292 59.01875, -102.33958 59.0...",104.0,311.0,1
1,71000092,402.158863,-102.334034,58.984292,99615.0,59.00,-102.50,0.085174,409,"POLYGON ((-102.37500 58.97125, -102.37708 58.9...",104.0,310.0,1
2,71007360,302.931320,-100.121819,56.987527,102512.0,57.00,-100.25,0.461395,316,"POLYGON ((-100.12500 56.92542, -100.12542 56.9...",112.0,319.0,1
3,71007360,302.931320,-100.121819,56.987527,102833.0,57.00,-100.00,0.538605,310,"POLYGON ((-100.10875 57.05625, -100.10458 57.0...",112.0,320.0,1
4,71011339,270.429858,-98.415879,56.020979,104763.0,56.00,-98.50,0.877963,269,"POLYGON ((-98.47625 56.06708, -98.47542 56.067...",116.0,326.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
964180,83034687,29.851234,-119.864094,69.335548,77104.0,69.25,-120.00,0.230905,287,"POLYGON ((-119.90208 69.34375, -119.90208 69.3...",63.0,240.0,1
964181,83034688,104.512570,-119.573274,69.283675,77746.0,69.25,-119.50,0.700302,175,"POLYGON ((-119.62500 69.31958, -119.61875 69.3...",63.0,242.0,1
964182,83034688,104.512570,-119.573274,69.283675,78067.0,69.25,-119.25,0.000679,117,"POLYGON ((-119.37500 69.29875, -119.37458 69.2...",63.0,243.0,1
964183,83034688,104.512570,-119.573274,69.283675,77425.0,69.25,-119.75,0.299019,232,"POLYGON ((-119.82292 69.34625, -119.82292 69.3...",63.0,241.0,1


In [201]:
# Perform check for success and update 'order' variable
for index, row in shp.iterrows():
    
    # Find where in the attributes file this ID_target occurs
    order = np.where(hru_lat_lon['ID_target'] == row['ID_target'])
    
    # Determine what to do with 'order'
    if order[0].size == 0:
        
        # Print a warning if the ID wasn't matched
        print(row['ID_target'], ' not found in the attributes.nc list of IDs')
    else:
        
        # Update the 'counter' field
        shp.at[index, 'order'] = order[0]+1

In [206]:
# Copy the shapefile and prepare for saving
test = pd.DataFrame(shp)

In [209]:
# Remove the actual shapes
del test['geometry']

In [210]:
test

,ID_target,MERIT_ele,lon_target,lat_target,ID_source,lat_source,lon_source,weight,ERA5_ele,rows,cols,order
0,71000092,402.158863,-102.334034,58.984292,99936.0,59.00,-102.25,0.914826,400,104.0,311.0,1
1,71000092,402.158863,-102.334034,58.984292,99615.0,59.00,-102.50,0.085174,409,104.0,310.0,1
2,71007360,302.931320,-100.121819,56.987527,102512.0,57.00,-100.25,0.461395,316,112.0,319.0,2
3,71007360,302.931320,-100.121819,56.987527,102833.0,57.00,-100.00,0.538605,310,112.0,320.0,2
4,71011339,270.429858,-98.415879,56.020979,104763.0,56.00,-98.50,0.877963,269,116.0,326.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
964180,83034687,29.851234,-119.864094,69.335548,77104.0,69.25,-120.00,0.230905,287,63.0,240.0,517313
964181,83034688,104.512570,-119.573274,69.283675,77746.0,69.25,-119.50,0.700302,175,63.0,242.0,517314
964182,83034688,104.512570,-119.573274,69.283675,78067.0,69.25,-119.25,0.000679,117,63.0,243.0,517314
964183,83034688,104.512570,-119.573274,69.283675,77425.0,69.25,-119.75,0.299019,232,63.0,241.0,517314


In [211]:
# Save the ordered file so that it can be re-used later
file_csv = 'remap_lapse.csv'
test.to_csv(file_csv) 